In [1]:
import json
import pandas as pd

In [2]:
files = ['gitlab_deploy_keys_and_tokens.json', 'gitlab_members.json', 'gitlab_project_export_import.json',
         'gitlab_project_webhooks.json', 'gitlab_projects.json', 'gitlab_repositories.json',
         'gitlab_user_keys.json', 'gitlab_user_tokens.json', 'gitlab_users.json',
         'gitlab_issues.json', 'gitlab_issue_notes.json'
]

In [3]:
unify_tags = {
    "repositories": "repository",
    "users": "user",
    "projects": "project",
    "members": "member",
    "deploy_keys": "deploy_key",
    "tokens": "token",
    "webhooks": "webhook",
    "deploy_tokens": "deploy_token",
    "keys": "key",
    "groups": "group",
    "hooks": "hook",
    "personal_access_tokens": "personal_access_token",
    "files": "file",
    "issues": "issue",
    "notes": "note"
}

In [4]:
def get_method_tag(method, data=None):
    if method == "GET":
        return "read"
    elif method == "POST":
        if data and "_method" in data:
            if data["_method"] in ["patch", "put"]:
                return "update"
            elif data["_method"] == "delete":
                return "delete"
        return "create"
    elif method == "PUT":
        return "update"
    elif method == "DELETE":
        return "delete"

In [5]:
def get_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    category = filename.split('/')[-1].split('.')[0]
    l = []
    for action in data:
        print(f"{category}, {action['action']}, {action['request']['url']}, {action['request']['method']}")

        semantic_string = action['action']
        request_url = action['request']['url']
        request_method = action['request']['method']
        request_data = {}
        if 'body' in action['request']:
            request_data = action['request']['body']
        regex = action['request'].get('regex', '')
        resource_types = action['request'].get('resource_types', [])
        splitted_endpoint = action['api']['endpoints'][0].split('/')
        tags = list(filter((lambda x: x != "" and not x.startswith(':')), splitted_endpoint))
        tags = [unify_tags.get(tag, tag) for tag in tags]    # Convert tags to unified names
        method_tag = get_method_tag(action['api']['method'], action['request'].get('body', {}))
        children = action['request'].get('children', [])
        example_urls = action['request'].get('example_urls', [])
        priority = action['request'].get('priority', 2)
        l.append([category, semantic_string, request_url, request_method, request_data, regex, resource_types, tags + [method_tag], children, example_urls, priority])
    return l

In [6]:
data = []
for file in files:
    file_data = get_data(f"./{file}")
    data.extend(file_data)

gitlab_deploy_keys_and_tokens, List deploy keys for project, https://gitlab.com/{group}/{project}/-/deploy_keys/*, GET
gitlab_deploy_keys_and_tokens, Add deploy key for a project, https://gitlab.com/{group}/{project}/-/deploy_keys, POST
gitlab_deploy_keys_and_tokens, Update deploy key, https://gitlab.com/{group}/{project}/-/deploy_keys/{key_id}, POST
gitlab_deploy_keys_and_tokens, Delete deploy key, https://gitlab.com/{group}/{project}/-/deploy_keys/{key_id}/disable, PUT
gitlab_deploy_keys_and_tokens, Create a project deploy token, https://gitlab.com/{group}/{project}/-/settings/repository/deploy_token/create, POST
gitlab_deploy_keys_and_tokens, Delete a project deploy token, https://gitlab.com/{group}/{project}/-/deploy_tokens/{token_id}/revoke, POST
gitlab_members, List all members of a group or project, https://gitlab.com/{group}/{project}/-/project_members, GET
gitlab_members, Add a member to a group or project, https://gitlab.com/api/v4/projects/{project_id}/invitations, POST
gitl

In [7]:
data

[['gitlab_deploy_keys_and_tokens',
  'List deploy keys for project',
  'https://gitlab.com/{group}/{project}/-/deploy_keys/*',
  'GET',
  {},
  '',
  [],
  ['project', 'deploy_key', 'read'],
  [],
  ['https://gitlab.com/macaroon777sweet/test/-/deploy_keys/enabled_keys?page=1&per_page=5',
   'https://gitlab.com/macaroon777sweet/test/-/deploy_keys/available_project_keys?page=1&per_page=5'],
  2],
 ['gitlab_deploy_keys_and_tokens',
  'Add deploy key for a project',
  'https://gitlab.com/{group}/{project}/-/deploy_keys',
  'POST',
  {},
  '',
  [],
  ['project', 'deploy_key', 'create'],
  [],
  [],
  2],
 ['gitlab_deploy_keys_and_tokens',
  'Update deploy key',
  'https://gitlab.com/{group}/{project}/-/deploy_keys/{key_id}',
  'POST',
  {},
  '',
  [],
  ['project', 'deploy_key', 'update'],
  [],
  [],
  2],
 ['gitlab_deploy_keys_and_tokens',
  'Delete deploy key',
  'https://gitlab.com/{group}/{project}/-/deploy_keys/{key_id}/disable',
  'PUT',
  {},
  '',
  [],
  ['project', 'deploy_key'

In [8]:
df = pd.DataFrame(data, columns=['category', 'semantic_string', 'request_url',
                                 'request_method', 'request_data', 'regex', 'resource_types', 'tags', 'children', 'example_urls', 'priority'])

In [9]:
df

,category,semantic_string,request_url,request_method,request_data,regex,resource_types,tags,children,example_urls,priority
0,gitlab_deploy_keys_and_tokens,List deploy keys for project,https://gitlab.com/{group}/{project}/-/deploy_...,GET,{},,[],"[project, deploy_key, read]",[],[https://gitlab.com/macaroon777sweet/test/-/de...,2
1,gitlab_deploy_keys_and_tokens,Add deploy key for a project,https://gitlab.com/{group}/{project}/-/deploy_...,POST,{},,[],"[project, deploy_key, create]",[],[],2
2,gitlab_deploy_keys_and_tokens,Update deploy key,https://gitlab.com/{group}/{project}/-/deploy_...,POST,{},,[],"[project, deploy_key, update]",[],[],2
3,gitlab_deploy_keys_and_tokens,Delete deploy key,https://gitlab.com/{group}/{project}/-/deploy_...,PUT,{},,[],"[project, deploy_key, delete]",[],[],2
4,gitlab_deploy_keys_and_tokens,Create a project deploy token,https://gitlab.com/{group}/{project}/-/setting...,POST,{},,[],"[project, deploy_token, create]",[],[],2
5,gitlab_deploy_keys_and_tokens,Delete a project deploy token,https://gitlab.com/{group}/{project}/-/deploy_...,POST,{},,[],"[project, deploy_token, delete]",[],[],2
6,gitlab_members,List all members of a group or project,https://gitlab.com/{group}/{project}/-/project...,GET,{},,[],"[project, member, read]","[{'method': 'GET', 'url': 'https://gitlab.com/...",[],2
7,gitlab_members,Add a member to a group or project,https://gitlab.com/api/v4/projects/{project_id...,POST,{},,[],"[project, member, create]",[],[],2
8,gitlab_members,Edit a member of a group or project,https://gitlab.com/{group}/{project}/-/project...,PUT,{},,[],"[project, member, update]",[],[],2
9,gitlab_members,Remove a member from a group or project,https://gitlab.com/{group}/{project}/-/project...,POST,{'_method': 'delete'},,[],"[project, member, delete]",[],[],2


In [10]:
read_api_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read"]
    },
    "description": "Grants read access, including all groups and projects, the container registry, and the package registry."
}

api_rule = {
    "effect": "allow",
    "match": "*",
    "description": "Grants complete read/write access, including all groups and projects, the container registry, the dependency proxy, and the package registry."
}


read_repository_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "repository"]
    },
    "description": "Grants read-only access to repositories on private projects."
}

write_repository_rule = {
    "effect": "allow",
    "match": {
        "tags": ["repository"],
    },
    "description": "Grants read-write access to repositories on private projects."
}

In [11]:
read_user_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "user", "~key", "~personal_access_token"]
    },
    "description": "Allow read access to user profile, except for sensitive information including keys and personal access tokens."
}

write_user_rule = {
    "effect": "allow",
    "match": {
        "tags": ["user", "~key", "~personal_access_token"]
    },
    "description": "Allow write access to user profile, except for sensitive information including keys and personal access tokens."
}

In [12]:
read_project_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "project", "~deploy_key", "~deploy_token"]
    },
    "description": "Allow read access to project resources, except for sensitive information including deploy keys and tokens."
}

write_project_rule = {
    "effect": "allow",
    "match": {
        "tags": ["project", "~deploy_key", "~deploy_token"]
    },
    "description": "Allow read/write access to project resources, except for sensitive information including deploy keys and tokens."
}

In [13]:
read_project_deploy_key_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "project", "deploy_key"]
    },
    "sensitive": True,
    "description": "Allow read access to project deploy keys."
}

write_project_deploy_key_rule = {
    "effect": "allow",
    "match": {
        "tags": ["project", "deploy_key"]
    },
    "sensitive": True,
    "description": "Allow write access to project deploy keys."
}

read_project_deploy_token_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "project", "deploy_token"]
    },
    "sensitive": True,
    "description": "Grants read access to project deploy tokens."
}

write_project_deploy_token_rule = {
    "effect": "allow",
    "match": {
        "tags": ["project", "deploy_token"]
    },
    "sensitive": True,
    "description": "Grants write access to project deploy tokens."
}

In [14]:
read_user_key_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "user", "key"]
    },
    "sensitive": True,
    "description": "Allow read access to user keys."
}

write_user_key_rule = {
    "effect": "allow",
    "match": {
        "tags": ["user", "key"]
    },
    "sensitive": True,
    "description": "Allow read/write access to user keys."
}

read_user_personal_access_token_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "user", "personal_access_token"]
    },
    "sensitive": True,
    "description": "Allow read access to user personal access tokens."
}

write_user_personal_access_token_rule = {
    "effect": "allow",
    "match": {
        "tags": ["user", "personal_access_token"]
    },
    "sensitive": True,
    "description": "Allow read/write access to user personal access tokens."
}

In [15]:
read_project_issue_rule = {
    "effect": "allow",
    "match": {
        "tags": ["read", "project", "issue"]
    },
    "description": "Allow read access to project issues."
}

write_project_issue_rule = {
    "effect": "allow",
    "match": {
        "tags": ["project", "issue"]
    },
    "description": "Allow read/write access to project issues."
}

In [16]:
def get_var_name(obj, namespace):
    return [name for name, val in namespace.items() if val is obj]
print(get_var_name(write_project_deploy_key_rule, globals()))

['write_project_deploy_key_rule']


In [17]:
rules = [
    read_api_rule,
    api_rule,
    read_repository_rule,
    write_repository_rule,
    read_user_rule,
    write_user_rule,
    read_project_rule,
    write_project_rule,
    read_project_deploy_key_rule,
    write_project_deploy_key_rule,
    read_project_deploy_token_rule,
    write_project_deploy_token_rule,
    read_user_key_rule,
    write_user_key_rule,
    read_user_personal_access_token_rule,
    write_user_personal_access_token_rule,
    read_project_issue_rule,
    write_project_issue_rule
]

# for rule in rules:
#     rule_name = get_var_name(rule, globals())[0].removesuffix('_rule')
#     with open(f"../rules/{rule_name}.json", 'w') as f:
#         json.dump(rule, f, indent=4)
    

In [18]:
def compose_policy(name: str, rules: list[dict]):
    policy = {
        "name": name,
        "domains": ["gitlab.com"],
        "default": "allow_public",
        "rules": rules,
    }
    return policy

In [19]:
df['request_data'] = df['request_data'].apply(lambda x: {k: v for k, v in x.items() if '{' not in v} if x else {})

sitemap_d = df.rename(columns={
    'request_url': 'url',
    'request_method': 'method',
    'tags': 'tags',
    'semantic_string': 'semantic_action',
    'request_data': 'body'
}).to_dict(orient='records')
sitemap_d

[{'category': 'gitlab_deploy_keys_and_tokens',
  'semantic_action': 'List deploy keys for project',
  'url': 'https://gitlab.com/{group}/{project}/-/deploy_keys/*',
  'method': 'GET',
  'body': {},
  'regex': '',
  'resource_types': [],
  'tags': ['project', 'deploy_key', 'read'],
  'children': [],
  'example_urls': ['https://gitlab.com/macaroon777sweet/test/-/deploy_keys/enabled_keys?page=1&per_page=5',
   'https://gitlab.com/macaroon777sweet/test/-/deploy_keys/available_project_keys?page=1&per_page=5'],
  'priority': 2},
 {'category': 'gitlab_deploy_keys_and_tokens',
  'semantic_action': 'Add deploy key for a project',
  'url': 'https://gitlab.com/{group}/{project}/-/deploy_keys',
  'method': 'POST',
  'body': {},
  'regex': '',
  'resource_types': [],
  'tags': ['project', 'deploy_key', 'create'],
  'children': [],
  'example_urls': [],
  'priority': 2},
 {'category': 'gitlab_deploy_keys_and_tokens',
  'semantic_action': 'Update deploy key',
  'url': 'https://gitlab.com/{group}/{pro

In [20]:
with open("../sitemap_tmp.json", "w") as f:
    json.dump(sitemap_d, f, indent=4)

In [21]:
from cellmate.policy.policy import Policy, Action
from cellmate.policy.sitemap import Sitemap

In [22]:
sitemap = Sitemap(sitemap_d)

In [23]:
policy_d = compose_policy("example_policy",[read_repository_rule, write_project_deploy_key_rule])
policy_d

{'name': 'example_policy',
 'domains': ['gitlab.com'],
 'default': 'allow_public',
 'rules': [{'effect': 'allow',
   'match': {'tags': ['read', 'repository']},
   'description': 'Grants read-only access to repositories on private projects.'},
  {'effect': 'allow',
   'match': {'tags': ['project', 'deploy_key']},
   'sensitive': True,
   'description': 'Allow write access to project deploy keys.'}]}

In [24]:
policy = Policy.from_dict(policy_d)

In [25]:
key_2_value = {
    "group": "macaroon777sweet-group",
    "project": "macaroon777sweet-project",
    "branch": "main",
    "filename": "README.md",
    "group_id": 1234,
    "project_id": 5678,
    'user_id': 9012,
    'key_id': 3456,
    'file_path': "test.txt",
    "dir_path": "subdir",
}

In [26]:
test_requests = sitemap_d.copy()
results = []
for row in test_requests:
    if 'url' in row:
        url = row['url']
        for key, value in key_2_value.items():
            url = url.replace(f"{{{key}}}", str(value))
        row['url'] = url
    action = Action.from_endpoint(
        url=row['url'],
        method=row['method'],
        body=row.get('body', {}),
        sitemap=sitemap
    )
    result = policy.evaluate(action)
    results.append(f"Action: {action.endpoint.method} {action.endpoint.url}, Result: {result}, Tags: {action.tags}")


⏳ Matching action: GET https://gitlab.com/macaroon777sweet-group/macaroon777sweet-project/-/deploy_keys/* with tags: ['project', 'deploy_key', 'read']
⏳ Matching action: GET https://gitlab.com/macaroon777sweet-group/macaroon777sweet-project/-/deploy_keys/* with tags: ['project', 'deploy_key', 'read']
🎯 Matched tags: ['project', 'deploy_key'] with action tags: ['project', 'deploy_key', 'read']
🔍 Evaluating against rule #1.
⏳ Matching action: POST https://gitlab.com/macaroon777sweet-group/macaroon777sweet-project/-/deploy_keys with tags: ['project', 'deploy_key', 'create']
⏳ Matching action: POST https://gitlab.com/macaroon777sweet-group/macaroon777sweet-project/-/deploy_keys with tags: ['project', 'deploy_key', 'create']
🎯 Matched tags: ['project', 'deploy_key'] with action tags: ['project', 'deploy_key', 'create']
🔍 Evaluating against rule #1.
⏳ Matching action: POST https://gitlab.com/macaroon777sweet-group/macaroon777sweet-project/-/deploy_keys/3456 with tags: ['project', 'deploy_key